<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/Stock_NeurIPS2018_SB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version** 



# Content

* [1. Task Description](#0)
* [2. Install Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. A List of Python Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download and Preprocess Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5. Build Market Environment in OpenAI Gym-style](#4)  
    * [5.1. Data Split](#4.1)  
    * [5.3. Environment for Training](#4.2)    
* [6. Train DRL Agents](#5)
* [7. Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
  

<a id='0'></a>
# Part 1. Task Discription

We train a DRL agent for stock trading. This task is modeled as a Markov Decision Process (MDP), and the objective function is maximizing (expected) cumulative return.

We specify the state-action-reward as follows:

* **State s**: The state space represents an agent's perception of the market environment. Just like a human trader analyzing various information, here our agent passively observes many features and learns by interacting with the market environment (usually by replaying historical data).

* **Action a**: The action space includes allowed actions that an agent can take at each state. For example, a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying. When an action operates multiple shares, a ∈{−k, ..., −1, 0, 1, ..., k}, e.g.. "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* **Reward function r(s, a, s′)**: Reward is an incentive for an agent to learn a better policy. For example, it can be the change of the portfolio value when taking a at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio values at state s′ and s, respectively


**Market environment**: 30 consituent stocks of Dow Jones Industrial Average (DJIA) index. Accessed at the starting date of the testing period.


The data for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Install Python Packages

<a id='1.1'></a>
## 2.1. Install packages



<a id='1.2'></a>
## 2.2. A list of Python packages 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [1]:
import datetime

In [137]:
import sysalpaca = tradeapi.REST(ALPACA_API_KEY,ALPACA_API_SECRET,ALPACA_API_BASE_URL, 'v2')
sys.path.append("../STOCK_DRL")

import warnings
warnings.filterwarnings("ignore")


import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

from Processed import get_processed_data

from module.yahoodownloader import YahooDownloader
from module.preprocessor import FeatureEngineer, data_split
from module.efficient_frontier import EfficientFrontier
from module import helper
from module.config_tickers import DOW_30_TICKER
from module.env_stocktrading import StockTradingEnv
from module.models import DRLAgent
from module.logger import configure
from module.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
    YESTERDAY,
    TODAY
)


<a id='1.4'></a>
## 2.4. Create Folders

In [3]:
from finrl.main import check_and_make_directories
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



<a id='2'></a>
# Part 3. Download Data
Yahoo Finance provides stock data, financial news, financial reports, etc. Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** in FinRL-Meta to fetch data via Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).



-----
class YahooDownloader:
    Retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()


In [4]:
# from config.py, TRAIN_START_DATE is a string
TRAIN_START_DATE
# from config.py, TRAIN_END_DATE is a string
TRAIN_END_DATE

'2020-05-16'

In [5]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [6]:
print(DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [7]:
df.shape

(104711, 8)

In [8]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.758536,746015200,AAPL,4
1,2009-01-02,58.590000,59.080002,57.750000,43.832626,6547900,AMGN,4
2,2009-01-02,18.570000,19.520000,18.400000,15.365308,10955700,AXP,4
3,2009-01-02,42.799999,45.560001,42.779999,33.941093,7010200,BA,4
4,2009-01-02,44.910000,46.980000,44.709999,31.579340,7117200,CAT,4


# Part 4: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

In [9]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3576, 8)
Successfully added vix
Successfully added turbulence index


In [10]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [11]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.758536,746015200.0,4.0,0.0,2.981388,2.652103,100.0,66.666667,100.0,2.758536,2.758536,39.189999,0.0
1,2009-01-02,AMGN,58.590000,59.080002,57.750000,43.832626,6547900.0,4.0,0.0,2.981388,2.652103,100.0,66.666667,100.0,43.832626,43.832626,39.189999,0.0
2,2009-01-02,AXP,18.570000,19.520000,18.400000,15.365308,10955700.0,4.0,0.0,2.981388,2.652103,100.0,66.666667,100.0,15.365308,15.365308,39.189999,0.0
3,2009-01-02,BA,42.799999,45.560001,42.779999,33.941093,7010200.0,4.0,0.0,2.981388,2.652103,100.0,66.666667,100.0,33.941093,33.941093,39.189999,0.0
4,2009-01-02,CAT,44.910000,46.980000,44.709999,31.579340,7117200.0,4.0,0.0,2.981388,2.652103,100.0,66.666667,100.0,31.579340,31.579340,39.189999,0.0
5,2009-01-02,CRM,8.025000,8.550000,7.912500,8.505000,4069200.0,4.0,0.0,2.981388,2.652103,100.0,66.666667,100.0,8.505000,8.505000,39.189999,0.0
6,2009-01-02,CSCO,16.410000,17.000000,16.250000,11.948333,40980600.0,4.0,0.0,2.981388,2.652103,100.0,66.666667,100.0,11.948333,11.948333,39.189999,0.0
7,2009-01-02,CVX,74.230003,77.300003,73.580002,43.677177,13695900.0,4.0,0.0,2.981388,2.652103,100.0,66.666667,100.0,43.677177,43.677177,39.189999,0.0
8,2009-01-02,DIS,22.760000,24.030001,22.500000,20.597494,9796600.0,4.0,0.0,2.981388,2.652103,100.0,66.666667,100.0,20.597494,20.597494,39.189999,0.0
9,2009-01-02,GS,84.019997,87.620003,82.190002,69.251762,14088500.0,4.0,0.0,2.981388,2.652103,100.0,66.666667,100.0,69.251762,69.251762,39.189999,0.0


In [12]:
mvo_df = processed_full.sort_values(['date','tic'],ignore_index=True)[['date','tic','close']]

In [13]:
int(len(mvo_df)/29)

3576

<a id='4'></a>
# Part 5. Build A Market Environment in OpenAI Gym-style
The training process involves observing stock price change, taking an action and reward's calculation. By interacting with the market environment, the agent will eventually derive a trading strategy that may maximize (expected) rewards.

Our market environment, based on OpenAI Gym, simulates stock markets with historical market data.

## Data Split
We split the data into training set and testing set as follows:

Training data period: 2009-01-01 to 2020-07-01

Trading data period: 2020-07-01 to 2021-10-31


In [14]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

82998
20706


In [15]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
2861,2020-05-15,UNH,292.410004,300.000000,290.720001,278.924500,12182800.0,4.0,5.160588,285.177010,264.528539,53.716923,79.362274,19.780788,268.314397,255.255441,31.889999,35.410001
2861,2020-05-15,V,179.979996,184.139999,178.869995,180.102737,10785900.0,4.0,3.517954,186.313232,157.702502,53.034253,99.128516,3.903157,169.547559,168.528678,31.889999,35.410001
2861,2020-05-15,VZ,54.630001,54.759998,54.209999,47.802040,10174000.0,4.0,-0.226676,51.578626,47.485447,46.826969,-165.232288,31.385117,49.595336,48.362977,31.889999,35.410001
2861,2020-05-15,WBA,38.310001,38.480000,37.500000,33.706383,17309300.0,4.0,-1.254167,40.409376,33.426905,40.887172,-199.332940,39.816185,37.336545,39.112128,31.889999,35.410001
2861,2020-05-15,WMT,123.470001,125.940002,123.089996,120.613297,10590000.0,4.0,0.498671,125.450061,114.250619,54.354134,-8.839888,0.410854,119.906015,114.327325,31.889999,35.410001


In [16]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2020-05-18,AAPL,78.292503,79.125000,77.580002,77.413994,135178400.0,0.0,2.628293,79.787532,65.136366,58.018325,128.081390,34.622794,70.754270,67.761639,29.299999,68.142649
0,2020-05-18,AMGN,243.660004,244.389999,231.800003,214.446991,5631400.0,0.0,4.237345,222.581972,208.394313,53.511967,57.692914,31.990941,211.469546,198.274529,29.299999,68.142649
0,2020-05-18,AXP,86.720001,89.000000,85.919998,85.229942,7781700.0,0.0,-1.096789,90.298286,74.832442,48.085958,33.068780,0.620002,83.116467,87.170145,29.299999,68.142649
0,2020-05-18,BA,126.180000,136.199997,125.800003,135.440002,43781800.0,0.0,-7.191083,142.655067,117.923934,42.534953,-29.016130,4.926572,135.367000,164.372832,29.299999,68.142649
0,2020-05-18,CAT,112.220001,115.660004,112.010002,108.081749,5603500.0,0.0,-1.169122,112.040900,96.223667,50.214111,17.694230,3.519003,106.144327,104.895562,29.299999,68.142649


In [17]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [18]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 291


In [19]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

## Environment for Training



In [20]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Train DRL Agents
* The DRL algorithms are from **Stable Baselines 3**. Users are also encouraged to try **ElegantRL** and **Ray RLlib**.
* FinRL includes fine-tuned standard DRL algorithms, such as DQN, DDPG, Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [21]:
agent = DRLAgent(env = env_train)

if_using_a2c = False
if_using_ddpg = False
if_using_ppo = False
if_using_td3 = False
if_using_sac = True


### Agent Training: 5 algorithms (A2C, DDPG, PPO, TD3, SAC)


### Agent 1: A2C


In [22]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)


{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device


In [23]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000) if if_using_a2c else None

### Agent 2: DDPG

In [24]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

if if_using_ddpg:
  # set up logger
  tmp_path = RESULTS_DIR + '/ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device


In [25]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000) if if_using_ddpg else None

### Agent 3: PPO

In [26]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device


In [27]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000) if if_using_ppo else None

### Agent 4: TD3

In [28]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

if if_using_td3:
  # set up logger
  tmp_path = RESULTS_DIR + '/td3'
  new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_td3.set_logger(new_logger_td3)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device


In [29]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000) if if_using_td3 else None

### Agent 5: SAC

In [30]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

if if_using_sac:
  # set up logger
  tmp_path = RESULTS_DIR + '/sac'
  new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_sac.set_logger(new_logger_sac)

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device


2023-03-21 14:11:15.954663: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 14:11:18.619174: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/acraf/.local/lib/python3.8/site-packages/cv2/../../lib64:
2023-03-21 14:11:18.619363: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/acraf/.local/lib/python3.8/site-packages/cv2/../../lib64:
20

Logging to results/sac


In [31]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=50000) if if_using_sac else None

----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 45        |
|    time_elapsed    | 252       |
|    total_timesteps | 11448     |
| train/             |           |
|    actor_loss      | 1.81e+03  |
|    critic_loss     | 1.41e+04  |
|    ent_coef        | 0.269     |
|    ent_coef_loss   | 90.4      |
|    learning_rate   | 0.0001    |
|    n_updates       | 11347     |
|    reward          | 1.5849121 |
----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 44        |
|    time_elapsed    | 517       |
|    total_timesteps | 22896     |
| train/             |           |
|    actor_loss      | 817       |
|    critic_loss     | 260       |
|    ent_coef        | 0.1       |
|    ent_coef_loss   | -99.6     |
|    learning_rate   | 0.0001    |
|    n_updates       | 22795     |
|    reward         

## In-sample Performance

Assume that the initial capital is $1,000,000.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data. If current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [32]:
data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [33]:
insample_risk_indicator.vix.describe()

count    2862.000000
mean       18.699046
std         8.440450
min         9.140000
25%        13.310000
50%        16.075000
75%        21.087500
max        82.690002
Name: vix, dtype: float64

In [34]:
insample_risk_indicator.vix.quantile(0.996)

57.49700183105472

In [35]:
insample_risk_indicator.turbulence.describe()

count    2862.000000
mean       34.582519
std        43.997250
min         0.000000
25%        14.908566
50%        24.046393
75%        39.165815
max       652.507573
Name: turbulence, dtype: float64

In [36]:
insample_risk_indicator.turbulence.quantile(0.996)

276.97425739806425

### Trading (Out-of-sample Performance)

We update periodically in order to take full advantage of the data, e.g., retrain quarterly, monthly or weekly. We also tune the parameters along the way, in this notebook we use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [37]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
env_trade, obs_trade = e_trade_gym.get_sb_env()

In [38]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2020-05-18,AAPL,78.292503,79.125000,77.580002,77.413994,135178400.0,0.0,2.628293,79.787532,65.136366,58.018325,128.081390,34.622794,70.754270,67.761639,29.299999,68.142649
0,2020-05-18,AMGN,243.660004,244.389999,231.800003,214.446991,5631400.0,0.0,4.237345,222.581972,208.394313,53.511967,57.692914,31.990941,211.469546,198.274529,29.299999,68.142649
0,2020-05-18,AXP,86.720001,89.000000,85.919998,85.229942,7781700.0,0.0,-1.096789,90.298286,74.832442,48.085958,33.068780,0.620002,83.116467,87.170145,29.299999,68.142649
0,2020-05-18,BA,126.180000,136.199997,125.800003,135.440002,43781800.0,0.0,-7.191083,142.655067,117.923934,42.534953,-29.016130,4.926572,135.367000,164.372832,29.299999,68.142649
0,2020-05-18,CAT,112.220001,115.660004,112.010002,108.081749,5603500.0,0.0,-1.169122,112.040900,96.223667,50.214111,17.694230,3.519003,106.144327,104.895562,29.299999,68.142649


In [39]:
trained_moedl = trained_a2c
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

AttributeError: 'NoneType' object has no attribute 'predict'

In [ ]:
trained_moedl = trained_ddpg
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

hit end!


In [ ]:
trained_moedl = trained_ppo
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

hit end!


In [ ]:
trained_moedl = trained_td3
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

hit end!


In [40]:
trained_moedl = trained_sac
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

hit end!


In [ ]:
df_account_value_a2c.shape

(714, 2)

<a id='7'></a>
# Part 6.5: Mean Variance Optimization

### Helper functions

### Calculate mean returns and variance-covariance matrix

<a id='6'></a>
# Part 7: Backtesting Results
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

In [ ]:
df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0])
df_result_ddpg = df_account_value_ddpg.set_index(df_account_value_ddpg.columns[0])
df_result_td3 = df_account_value_td3.set_index(df_account_value_td3.columns[0])
df_result_ppo = df_account_value_ppo.set_index(df_account_value_ppo.columns[0])
df_result_sac = df_account_value_sac.set_index(df_account_value_sac.columns[0])

result = pd.merge(df_result_a2c, df_result_ddpg, left_index=True, right_index=True)
result = pd.merge(result, df_result_td3, left_index=True, right_index=True)
result = pd.merge(result, df_result_ppo, left_index=True, right_index=True)
result = pd.merge(result, df_result_sac, left_index=True, right_index=True)
result.columns = ['a2c', 'ddpg', 'td3', 'ppo', 'sac']

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)
plt.figure()
result.plot()
plt.savefig("trained_models/models_" + ".jpg")

In [ ]:
result.tail(1)

,a2c,ddpg,td3,ppo,sac
date,,,,,
2023-03-17,1.286389e+06,1.266192e+06,1.506513e+06,1.286005e+06,1.448611e+06


In [ ]:

now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_sac = helper.backtest_stats(account_value = df_account_value_sac)
perf_stats_sac = pd.DataFrame(perf_stats_sac)

perf_stats_ddpg = helper.backtest_stats(account_value = df_account_value_ddpg)
perf_stats_ddpg = pd.DataFrame(perf_stats_ddpg)

perf_stats_ppo = helper.backtest_stats(account_value = df_account_value_ppo)
perf_stats_ppo = pd.DataFrame(perf_stats_ppo)

perf_stats_td3 = helper.backtest_stats(account_value = df_account_value_td3)
perf_stats_td3 = pd.DataFrame(perf_stats_td3)

perf_stats_a2c = helper.backtest_stats(account_value = df_account_value_a2c)
perf_stats_a2c = pd.DataFrame(perf_stats_a2c)



Annual return          0.139742
Cumulative returns     0.448611
Annual volatility      0.168389
Sharpe ratio           0.862391
Calmar ratio           0.821600
Stability              0.763056
Max drawdown          -0.170085
Omega ratio            1.155094
Sortino ratio          1.217187
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.987886
Daily value at risk   -0.020639
dtype: float64
Annual return          0.086867
Cumulative returns     0.266192
Annual volatility      0.208057
Sharpe ratio           0.505201
Calmar ratio           0.298338
Stability              0.124270
Max drawdown          -0.291169
Omega ratio            1.089099
Sortino ratio          0.709201
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.034249
Daily value at risk   -0.025796
dtype: float64
Annual return          0.092839
Cumulative returns     0.286005
Annual volatility      0.168020
Sharpe ratio           0.613177
Calmar rat

In [ ]:
baseline_df = YahooDownloader(
        ticker_list =["^DJI"], 
        start_date = TRADE_START_DATE,
        end_date = TRADE_END_DATE).fetch_data()

stats = helper.backtest_stats(baseline_df, value_col_name = 'close')

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (714, 8)
Annual return          0.095633
Cumulative returns     0.295341
Annual volatility      0.172516
Sharpe ratio           0.616696
Calmar ratio           0.435867
Stability              0.316811
Max drawdown          -0.219408
Omega ratio            1.110738
Sortino ratio          0.862038
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.018332
Daily value at risk   -0.021313
dtype: float64


In [ ]:

Annual_return_sac = perf_stats_sac.T['Annual return'][0]
Annual_return_ppo = perf_stats_ppo.T['Annual return'][0]
Annual_return_td3 = perf_stats_td3.T['Annual return'][0]
Annual_return_ddpg = perf_stats_ddpg.T['Annual return'][0]
Annual_return_a2c = perf_stats_a2c.T['Annual return'][0]
Annual_return = stats[0]

In [ ]:
d = {'sac':Annual_return_sac, 'a2c':Annual_return_a2c, 'td3':Annual_return_td3, 'ddpg': Annual_return_ddpg, 'ppo':Annual_return_ppo}
best_model = max(d.items(), key=lambda i: i[1])

In [ ]:
test = pd.concat([train, trade], ignore_index=True)

In [ ]:
test

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.758535,746015200.0,4.0,0.000000,2.981390,2.652101,100.000000,66.666667,100.000000,2.758535,2.758535,39.189999,0.000000
1,2009-01-02,AMGN,58.590000,59.080002,57.750000,43.832623,6547900.0,4.0,0.000000,2.981390,2.652101,100.000000,66.666667,100.000000,43.832623,43.832623,39.189999,0.000000
2,2009-01-02,AXP,18.570000,19.520000,18.400000,15.365305,10955700.0,4.0,0.000000,2.981390,2.652101,100.000000,66.666667,100.000000,15.365305,15.365305,39.189999,0.000000
3,2009-01-02,BA,42.799999,45.560001,42.779999,33.941086,7010200.0,4.0,0.000000,2.981390,2.652101,100.000000,66.666667,100.000000,33.941086,33.941086,39.189999,0.000000
4,2009-01-02,CAT,44.910000,46.980000,44.709999,31.579329,7117200.0,4.0,0.000000,2.981390,2.652101,100.000000,66.666667,100.000000,31.579329,31.579329,39.189999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103699,2023-03-17,UNH,471.390015,472.010010,466.299988,469.500000,5283100.0,4.0,-5.674475,495.748808,455.035442,43.201078,-74.670654,17.973473,478.157824,488.492234,25.510000,36.582472
103700,2023-03-17,V,214.869995,218.660004,214.869995,217.389999,18797500.0,4.0,-1.688439,225.869792,214.026209,48.505392,-88.589882,31.538839,222.927316,220.309803,25.510000,36.582472
103701,2023-03-17,VZ,36.549999,36.830002,36.160000,36.790001,47335300.0,4.0,-0.845187,40.137241,35.792759,40.341567,-116.794477,33.516601,38.795000,39.439228,25.510000,36.582472
103702,2023-03-17,WBA,33.150002,33.340000,32.529999,32.910000,17223100.0,4.0,-0.813598,37.304085,32.289915,39.672778,-149.920105,44.977622,35.261483,35.889767,25.510000,36.582472


In [185]:
from module.models import DRLAgent
from module.env_stocktrading import StockTradingEnv
import time
import pandas as pd
import numpy as np
import alpaca_trade_api as tradeapi
import datetime
import threading

from module.processor_alpaca import AlpacaProcessor
from module.config_tickers import DOW_30_TICKER

from module.config import (
    ALPACA_API_BASE_URL,
    ALPACA_API_KEY,
    ALPACA_API_SECRET,
    INDICATORS
)

class Alpaca():

    def __init__(self, model, turbulence_thresh=70, time_interval=60):
        self.model = model
        self.turbulence_thresh = turbulence_thresh
        self.time_interval = time_interval
        try:
            self.alpaca = tradeapi.REST(ALPACA_API_KEY,ALPACA_API_SECRET,ALPACA_API_BASE_URL, 'v2')
        except:
            raise ValueError('Fail to connect Alpaca. Please check account info and internet connection.')
        
        self.stocks = np.asarray([0] * len(DOW_30_TICKER)) #stocks holding
        self.stocks_cd = np.zeros_like(self.stocks) 
        self.cash = None #cash record 
        self.stocks_df = pd.DataFrame(self.stocks, columns=['stocks'], index = DOW_30_TICKER)
        self.asset_list = []
        self.price = np.asarray([0] * len(DOW_30_TICKER))
        self.stockUniverse = DOW_30_TICKER
        self.turbulence_bool = 0
        self.equities = []

    def run(self):
        orders = self.alpaca.list_orders(status="open")
        for order in orders:
          self.alpaca.cancel_order(order.id)
    
        # Wait for market to open.
        print("Waiting for market to open...")
        tAMO = threading.Thread(target=self.awaitMarketOpen)
        tAMO.start()
        tAMO.join()
        print("Market opened.")
    
        while True:
          
          # Figure out when the market will close so we can prepare to sell beforehand.
          clock = self.alpaca.get_clock()
          closingTime = clock.next_close.replace(tzinfo=datetime.timezone.utc).timestamp()
          currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
          self.timeToClose = closingTime - currTime

          if(self.timeToClose < (60)):
            # Close all positions when 1 minutes til market close.
            print("Market closing soon. Stop trading.")
            break

          else:
            trade = threading.Thread(target=self.trade)
            trade.start()
            trade.join()
            last_equity = float(self.alpaca.get_account().last_equity)
            cur_time = time.time()
            self.equities.append([cur_time,last_equity])
            time.sleep(self.time_interval)
    
    def awaitMarketOpen(self):
        isOpen = self.alpaca.get_clock().is_open

        while(not isOpen):
            clock = self.alpaca.get_clock()
            openingTime = clock.next_open.replace(tzinfo=datetime.timezone.utc).timestamp()
            currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
            timeToOpen = int((openingTime - currTime) / 60)
            print(str(timeToOpen) + " minutes til market open.")
            time.sleep(60)
            isOpen = self.alpaca.get_clock().is_open


    def submitOrder(self, qty, stock, side, resp):
        if(qty > 0):
          try:
            self.ae_trainlpaca.submit_order(stock, qty, side, "market", "day")
            print("Market order of | " + str(qty) + " " + stock + " " + side + " | completed.")
            resp.append(True)
          except:
            print("Order of | " + str(qty) + " " + stock + " " + side + " | did not go through.")
            resp.append(False)
        else:
          print("Quantity is 0, order of | " + str(qty) + " " + stock + " " + side + " | not completed.")
          resp.append(True)
    
    def trade(self):
        state = self.get_state()

        action = self.model.predict(state)[0]

        if self.turbulence_bool == 0:
            min_action = 10  # stock_cd
            for index in np.where(action < -min_action)[0]:  # sell_index:
                sell_num_shares = min(self.stocks[index], -action[index])
                qty =  abs(int(sell_num_shares))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, self.stockUniverse[index], 'sell', respSO))
                tSubmitOrder.start()
                tSubmitOrder.join()
                self.cash = float(self.alpaca.get_account().cash)
                self.stocks_cd[index] = 0

            for index in np.where(action > min_action)[0]:  # buy_index:
                if self.cash < 0:
                    tmp_cash = 0
                else:
                    tmp_cash = self.cash
                buy_num_shares = min(tmp_cash // self.price[index], abs(int(action[index])))
                if (buy_num_shares != buy_num_shares): # if buy_num_change = nan
                    qty = 0 # set to 0 quantity
                else:
                    qty = abs(int(buy_num_shares))
                qty = abs(int(buy_num_shares))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, self.stockUniverse[index], 'buy', respSO))
                tSubmitOrder.start()
                tSubmitOrder.join()
                self.cash = float(self.alpaca.get_account().cash)
                self.stocks_cd[index] = 0
                
        else:  # sell all when turbulence
            positions = self.alpaca.list_positions()
            for position in positions:
                if(position.side == 'long'):
                    orderSide = 'sell'
                else:
                    orderSide = 'buy'
                qty = abs(int(float(position.qty)))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, position.symbol, orderSide, respSO))
                tSubmitOrder.start()
                tSubmitOrder.join()
            
            self.stocks_cd[:] = 0    

           
    def get_state(self):
       alpaca = AlpacaProcessor(api=self.alpaca)
       price, tech, turbulence = alpaca.fetch_latest_data(ticker_list = DOW_30_TICKER, time_interval='1Min',
                                                     tech_indicator_list=INDICATORS)
       


       turbulence_bool = 1 if turbulence >= self.turbulence_thresh else 0
        
       turbulence = (self.sigmoid_sign(turbulence, self.turbulence_thresh) * 2 ** -5).astype(np.float32)
        
       tech = tech * 2 ** -7
       positions = self.alpaca.list_positions()
       stocks = [0] * len(DOW_30_TICKER)
       for position in positions:
          ind = DOW_30_TICKER.index(position.symbol)
          stocks[ind] = ( abs(int(float(position.qty))))
        
       stocks = np.asarray(stocks, dtype = float)
       cash = float(self.alpaca.get_account().cash)
       self.cash = cash
       self.stocks = stocks
       self.turbulence_bool = turbulence_bool 
       self.price = price
        
        
        
       amount = np.array(self.cash * (2 ** -12), dtype=np.float32)
       scale = np.array(2 ** -6, dtype=np.float32)
       state = np.hstack((amount,
                    turbulence,
                    self.turbulence_bool,
                    price * scale,
                    self.stocks * scale,
                    self.stocks_cd,
                    tech,
                    )).astype(np.float32)
       state[np.isnan(state)] = 0.0
       state[np.isinf(state)] = 0.0
       return state
    
    @staticmethod
    def sigmoid_sign(ary, thresh):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x * np.e)) - 0.5

        return sigmoid(ary / thresh) * thresh

In [186]:
test = Alpaca(model=trained_sac)

In [ ]:
test.run()

In [196]:
import alpaca_trade_api as tradeapi

In [197]:
data = pd.DataFrame()
test = tradeapi.REST(ALPACA_API_KEY,ALPACA_API_SECRET,ALPACA_API_BASE_URL, 'v2')


In [198]:
d = test.get_latest_bars(DOW_30_TICKER)

In [218]:
d2 = d['AAPL']

In [232]:
str(d2)

"BarV2({   'c': 157.84,\n    'h': 157.84,\n    'l': 157.82,\n    'n': 15,\n    'o': 157.82,\n    't': '2023-03-21T16:09:00Z',\n    'v': 2243,\n    'vw': 157.827102})"

In [ ]:
alpaca = l = []
l.append(today)
for i in price:
    l.append(i)
for i in tech:
    l.append(i)
for i in turbulence:
    l.append(i)

In [194]:
alpaca2.get_latest_bar('AAPL')

BarV2({   'c': 157.51,
    'h': 157.68,
    'l': 157.51,
    'n': 22,
    'o': 157.68,
    't': '2023-03-21T16:04:00Z',
    'v': 2246,
    'vw': 157.603515})